# Sherlock 데이터 전처리 확인 및 BSDS 피팅 준비

이 노트북은 다음을 확인합니다:
1. Sherlock 데이터 다운로드 및 로드
2. 데이터 형태 및 품질 검증
3. BSDS 모델에 맞는 전처리 수행
4. Human boundary와의 비교를 위한 준비

**워크플로우**: 로컬 코드 생성 → git push → 서버에서 실행

## 0. 환경 설정

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# BSDS modules
from hmm_baseline.data_loaders import (
    load_sherlock_figshare,
    get_sherlock_human_boundaries,
    SHERLOCK_HUMAN_BOUNDARIES,
    DatasetInfo
)
from bsds_complete.utils.data_utils import validate_input, preprocess_data, concatenate_subjects

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print(f"Project root: {project_root}")
print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")

## 1. Sherlock 데이터셋 정보 확인

In [ ]:
# Sherlock 데이터셋 정보
sherlock_info = DatasetInfo.SHERLOCK

print("=" * 60)
print("Sherlock Dataset Information")
print("=" * 60)
for key, value in sherlock_info.items():
    print(f"{key:25s}: {value}")

print("\n" + "=" * 60)
print("Human-Annotated Boundaries")
print("=" * 60)
human_info = get_sherlock_human_boundaries(TR=1.5)
print(f"Number of boundaries: {human_info['n_boundaries']}")
print(f"TR: {human_info['TR']} seconds")
print(f"First 5 TRs: {human_info['tr_indices'][:5]}")
print(f"First 5 timestamps (sec): {human_info['timestamps'][:5]}")

## 2. 데이터 다운로드 상태 확인

### 다운로드 방법 (서버에서 실행)
```bash
mkdir -p ~/data/sherlock
cd ~/data/sherlock

# Figshare에서 다운로드 (약 500MB)
wget -O sherlock_atlas_data.zip "https://figshare.com/ndownloader/files/9021937"
unzip sherlock_atlas_data.zip
```

In [ ]:
# 데이터 경로 설정 (서버 환경에 맞게 수정)
SHERLOCK_DATA_DIR = Path.home() / "data" / "sherlock"

# 대체 경로들 시도
alternative_paths = [
    Path.home() / "data" / "sherlock",
    Path("/storage/bigdata/sherlock"),
    Path("/scratch/sherlock"),
    project_root.parent / "data" / "sherlock",
]

data_dir = None
for path in alternative_paths:
    if path.exists():
        data_dir = path
        print(f"✅ Found data directory: {data_dir}")
        break

if data_dir is None:
    print("❌ Sherlock data directory not found.")
    print("\nPlease download data using the following commands:")
    print("""    
    mkdir -p ~/data/sherlock
    cd ~/data/sherlock
    wget -O sherlock_atlas_data.zip "https://figshare.com/ndownloader/files/9021937"
    unzip sherlock_atlas_data.zip
    """)
else:
    # 파일 목록 확인
    npy_files = list(data_dir.glob("*.npy"))
    print(f"\nFound {len(npy_files)} .npy files:")
    for f in sorted(npy_files):
        size_mb = f.stat().st_size / (1024 * 1024)
        print(f"  - {f.name}: {size_mb:.1f} MB")

## 3. 데이터 로드 및 형태 확인

In [ ]:
def check_sherlock_data(data_dir: Path):
    """Sherlock 데이터 검증 함수"""
    if not data_dir or not data_dir.exists():
        print("❌ Data directory not available. Skipping data check.")
        return None
    
    # ROI 별 데이터 로드
    roi_list = ['AG', 'PMC', 'EAC', 'PPA', 'PCC']
    data_info = {}
    
    print("\n" + "=" * 60)
    print("Loading Sherlock ROI Data")
    print("=" * 60)
    
    for roi in roi_list:
        roi_file = data_dir / f"{roi}_movie_1TR.npy"
        if not roi_file.exists():
            roi_file = data_dir / f"{roi}.npy"
        
        if roi_file.exists():
            data = np.load(roi_file)
            data_info[roi] = {
                'file': roi_file.name,
                'shape': data.shape,
                'dtype': str(data.dtype),
                'min': float(np.min(data)),
                'max': float(np.max(data)),
                'mean': float(np.mean(data)),
                'std': float(np.std(data)),
                'nan_count': int(np.sum(np.isnan(data))),
                'inf_count': int(np.sum(np.isinf(data))),
                'data': data
            }
            print(f"\n✅ {roi}:")
            print(f"   Shape: {data.shape}")
            print(f"   Stats: min={data_info[roi]['min']:.3f}, max={data_info[roi]['max']:.3f}")
            print(f"          mean={data_info[roi]['mean']:.3f}, std={data_info[roi]['std']:.3f}")
            print(f"   NaN/Inf: {data_info[roi]['nan_count']}/{data_info[roi]['inf_count']}")
        else:
            print(f"❌ {roi}: File not found")
    
    return data_info

# 데이터 로드
data_info = check_sherlock_data(data_dir)

## 4. 데이터 형태 분석 및 BSDS 호환성 확인

In [ ]:
def analyze_data_for_bsds(data_info: dict):
    """BSDS 모델에 대한 데이터 호환성 분석"""
    if data_info is None:
        print("❌ No data to analyze")
        return
    
    print("\n" + "=" * 60)
    print("BSDS Compatibility Analysis")
    print("=" * 60)
    
    # BSDS expects: List of (D x T) arrays
    # D = number of ROIs/dimensions
    # T = number of time points
    
    for roi, info in data_info.items():
        if 'data' not in info:
            continue
            
        data = info['data']
        shape = data.shape
        
        print(f"\n{roi}:")
        print(f"  Original shape: {shape}")
        
        if data.ndim == 3:
            # (n_subjects, n_timepoints, n_voxels)
            n_subjects, n_timepoints, n_voxels = shape
            print(f"  Interpretation: {n_subjects} subjects × {n_timepoints} TRs × {n_voxels} voxels")
            print(f"  ✅ Multi-subject format detected")
            
            # BSDS format: List of (D x T) arrays
            # 여기서 D = n_voxels, T = n_timepoints
            print(f"  BSDS format: List of ({n_voxels} × {n_timepoints}) arrays, {n_subjects} subjects")
            
        elif data.ndim == 2:
            # Could be (n_timepoints, n_voxels) or (n_voxels, n_timepoints)
            print(f"  Could be: ({shape[0]} TRs × {shape[1]} voxels) OR ({shape[0]} voxels × {shape[1]} TRs)")
            
            # Sherlock has ~1976 TRs (48 min / 1.5s)
            if shape[0] > 1900 and shape[0] < 2000:
                print(f"  ⚠️ Likely (TRs × voxels) - needs transpose for BSDS")
            elif shape[1] > 1900 and shape[1] < 2000:
                print(f"  ✅ Likely (voxels × TRs) - correct for BSDS")
        
        # Data quality checks
        print(f"  Quality:")
        print(f"    - NaN values: {info['nan_count']} ({'❌ FIX REQUIRED' if info['nan_count'] > 0 else '✅ OK'})")
        print(f"    - Inf values: {info['inf_count']} ({'❌ FIX REQUIRED' if info['inf_count'] > 0 else '✅ OK'})")
        
        # Check if z-scored
        is_zscored = abs(info['mean']) < 0.1 and 0.5 < info['std'] < 2.0
        print(f"    - Z-scored: {'✅ Yes (likely)' if is_zscored else '⚠️ No (preprocessing needed)'}")

analyze_data_for_bsds(data_info)

## 5. BSDS용 데이터 전처리

In [ ]:
def prepare_for_bsds(data_info: dict, roi: str = 'AG') -> list:
    """
    BSDS 모델용 데이터 준비
    
    BSDS expects: List of (D x T) arrays
    - D = number of dimensions (ROIs/voxels)
    - T = number of time points
    
    Returns:
        List of preprocessed data arrays, one per subject
    """
    if data_info is None or roi not in data_info:
        print(f"❌ Data for ROI '{roi}' not available")
        return None
    
    data = data_info[roi]['data']
    
    print(f"\n{'=' * 60}")
    print(f"Preparing {roi} data for BSDS")
    print(f"{'=' * 60}")
    print(f"Original shape: {data.shape}")
    
    # Convert to list of (D x T) arrays
    data_list = []
    
    if data.ndim == 3:
        # (n_subjects, n_timepoints, n_voxels)
        n_subjects = data.shape[0]
        for i in range(n_subjects):
            # Transpose to (D x T) = (n_voxels x n_timepoints)
            subj_data = data[i, :, :].T  # (V x T)
            data_list.append(subj_data)
            
    elif data.ndim == 2:
        # Single subject or need to infer orientation
        if data.shape[0] > 1900 and data.shape[0] < 2000:
            # (T x D) -> transpose to (D x T)
            data_list.append(data.T)
        else:
            # Already (D x T)
            data_list.append(data)
    
    # Apply preprocessing to each subject
    print(f"\nApplying preprocessing...")
    preprocessed_list = []
    
    for i, d in enumerate(data_list):
        # Use BSDS preprocess function
        d_preprocessed = preprocess_data(d, standardize=True, detrend=True)
        preprocessed_list.append(d_preprocessed)
        
        print(f"  Subject {i+1}: {d.shape} -> preprocessed")
        print(f"    Mean: {np.mean(d_preprocessed):.6f}, Std: {np.std(d_preprocessed):.3f}")
    
    # Validate using BSDS utility
    print(f"\nValidating BSDS input format...")
    try:
        n_dims, n_samples_list, n_subjects = validate_input(preprocessed_list)
        print(f"✅ Valid BSDS input!")
        print(f"   Dimensions (D): {n_dims}")
        print(f"   Subjects: {n_subjects}")
        print(f"   Samples per subject: {n_samples_list}")
    except Exception as e:
        print(f"❌ Validation failed: {e}")
        return None
    
    return preprocessed_list

# AG ROI로 전처리
if data_info is not None and 'AG' in data_info:
    bsds_data = prepare_for_bsds(data_info, roi='AG')
else:
    print("⚠️ Skipping preprocessing - data not loaded")
    bsds_data = None

## 6. 데이터 시각화

In [ ]:
def visualize_sherlock_data(data_list: list, human_boundaries: np.ndarray, TR: float = 1.5):
    """Sherlock 데이터 및 human boundary 시각화"""
    if data_list is None:
        print("❌ No data to visualize")
        return
    
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    
    # Use first subject
    data = data_list[0]
    D, T = data.shape
    time_min = np.arange(T) * TR / 60  # Convert to minutes
    
    # 1. Time series heatmap
    ax = axes[0]
    im = ax.imshow(data, aspect='auto', cmap='RdBu_r', 
                   extent=[0, time_min[-1], D, 0],
                   vmin=-3, vmax=3)
    ax.set_xlabel('Time (minutes)')
    ax.set_ylabel('Voxel')
    ax.set_title(f'Sherlock fMRI Data - Subject 1 (D={D}, T={T})')
    plt.colorbar(im, ax=ax, label='Z-scored BOLD')
    
    # Add human boundaries
    for hb in human_boundaries:
        ax.axvline(hb * TR / 60, color='lime', linewidth=1, alpha=0.7)
    
    # 2. Mean time series with boundaries
    ax = axes[1]
    mean_ts = np.mean(data, axis=0)
    ax.plot(time_min, mean_ts, 'b-', linewidth=0.5, alpha=0.7)
    ax.fill_between(time_min, mean_ts - np.std(data, axis=0), 
                    mean_ts + np.std(data, axis=0), alpha=0.3)
    
    # Add boundaries
    for hb in human_boundaries:
        ax.axvline(hb * TR / 60, color='green', linewidth=1, alpha=0.7)
    
    ax.set_xlabel('Time (minutes)')
    ax.set_ylabel('Mean BOLD (z-score)')
    ax.set_title('Mean Time Series with Human-Annotated Event Boundaries (green)')
    ax.legend(['Mean ± Std', 'Human Boundaries'], loc='upper right')
    
    # 3. Event duration histogram
    ax = axes[2]
    event_starts = np.concatenate([[0], human_boundaries])
    event_durations = np.diff(np.concatenate([event_starts, [T]])) * TR
    
    ax.hist(event_durations, bins=15, edgecolor='black', alpha=0.7)
    ax.axvline(np.mean(event_durations), color='red', linestyle='--', 
               label=f'Mean: {np.mean(event_durations):.1f}s')
    ax.axvline(np.median(event_durations), color='orange', linestyle='--',
               label=f'Median: {np.median(event_durations):.1f}s')
    ax.set_xlabel('Event Duration (seconds)')
    ax.set_ylabel('Count')
    ax.set_title('Distribution of Human-Annotated Event Durations')
    ax.legend()
    
    plt.tight_layout()
    plt.savefig('sherlock_data_overview.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Print statistics
    print(f"\n{'=' * 60}")
    print("Event Duration Statistics")
    print(f"{'=' * 60}")
    print(f"Number of events: {len(event_durations)}")
    print(f"Mean duration: {np.mean(event_durations):.1f}s ({np.mean(event_durations)/TR:.1f} TRs)")
    print(f"Median duration: {np.median(event_durations):.1f}s")
    print(f"Std duration: {np.std(event_durations):.1f}s")
    print(f"Min/Max: {np.min(event_durations):.1f}s / {np.max(event_durations):.1f}s")

if bsds_data is not None:
    visualize_sherlock_data(bsds_data, SHERLOCK_HUMAN_BOUNDARIES, TR=1.5)
else:
    print("⚠️ Skipping visualization - data not available")

## 7. BSDS 피팅 준비 확인

In [ ]:
def bsds_readiness_check(data_list: list):
    """BSDS 피팅을 위한 최종 준비 상태 확인"""
    print("\n" + "=" * 60)
    print("BSDS Fitting Readiness Check")
    print("=" * 60)
    
    checks = {
        'data_loaded': data_list is not None,
        'is_list': isinstance(data_list, list) if data_list else False,
        'correct_dims': False,
        'no_nan': False,
        'no_inf': False,
        'standardized': False,
    }
    
    if data_list is not None and len(data_list) > 0:
        # Check dimensions
        all_2d = all(d.ndim == 2 for d in data_list)
        all_same_d = len(set(d.shape[0] for d in data_list)) == 1
        checks['correct_dims'] = all_2d and all_same_d
        
        # Check NaN/Inf
        checks['no_nan'] = not any(np.any(np.isnan(d)) for d in data_list)
        checks['no_inf'] = not any(np.any(np.isinf(d)) for d in data_list)
        
        # Check standardization
        concat = np.hstack(data_list)
        checks['standardized'] = abs(np.mean(concat)) < 0.1 and 0.5 < np.std(concat) < 2.0
    
    # Print results
    all_passed = True
    for check, passed in checks.items():
        status = '✅' if passed else '❌'
        print(f"  {status} {check.replace('_', ' ').title()}: {passed}")
        if not passed:
            all_passed = False
    
    print("\n" + "-" * 60)
    if all_passed:
        print("🎉 All checks passed! Data is ready for BSDS fitting.")
        print("\nNext steps:")
        print("  1. Push code to git: git add . && git commit -m 'add sherlock preprocessing' && git push")
        print("  2. On server: git pull && python run_sherlock_bsds.py")
    else:
        print("⚠️ Some checks failed. Please fix issues before fitting.")
    
    return all_passed

bsds_readiness_check(bsds_data)

## 8. BSDS 피팅 테스트 (소규모)

In [ ]:
def test_bsds_fitting(data_list: list, max_subjects: int = 2, n_iter: int = 10):
    """BSDS 모델 피팅 테스트 (소규모)"""
    if data_list is None:
        print("❌ No data for testing")
        return None
    
    from bsds_complete.core.model import BSDSModel
    from bsds_complete.core.config import BSDSConfig
    
    # Use subset of subjects for quick test
    test_data = data_list[:max_subjects]
    
    # Subsample time points for faster testing
    test_data_small = [d[:, ::4] for d in test_data]  # Every 4th time point
    
    print(f"\n{'=' * 60}")
    print("BSDS Fitting Test")
    print(f"{'=' * 60}")
    print(f"Subjects: {len(test_data_small)}")
    print(f"Dimensions: {test_data_small[0].shape[0]}")
    print(f"Time points: {[d.shape[1] for d in test_data_small]}")
    
    # Configure model
    config = BSDSConfig(
        n_states=5,
        max_ldim=3,
        n_iter=n_iter,
        n_init_learning=1,
        n_init_iter=5,
        TR=1.5,
        verbose=True,
        random_seed=42
    )
    
    print(f"\nModel config:")
    print(f"  n_states: {config.n_states}")
    print(f"  max_ldim: {config.max_ldim}")
    print(f"  n_iter: {config.n_iter}")
    
    # Fit model
    print(f"\nFitting model...")
    try:
        model = BSDSModel(config)
        model.fit(test_data_small)
        
        print(f"\n✅ Model fitted successfully!")
        
        # Get results
        states = model.get_states()
        stats = model.get_summary_statistics()
        
        print(f"\nResults:")
        print(f"  Effective states: {stats.get('effective_n_states', 'N/A')}")
        print(f"  State occupancy: {stats.get('occupancy', 'N/A')}")
        print(f"  Final log-likelihood: {model.log_lik_history[-1]:.2f}")
        
        return model, states, stats
        
    except Exception as e:
        print(f"❌ Fitting failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# 테스트 실행 (데이터가 있는 경우에만)
if bsds_data is not None:
    test_result = test_bsds_fitting(bsds_data, max_subjects=2, n_iter=10)
else:
    print("⚠️ Skipping BSDS test - data not available")

## 9. 저장 및 다음 단계

In [ ]:
def save_preprocessed_data(data_list: list, output_dir: Path, roi: str = 'AG'):
    """전처리된 데이터 저장"""
    if data_list is None:
        print("❌ No data to save")
        return
    
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    output_file = output_dir / f"sherlock_{roi}_preprocessed.npy"
    
    # Save as object array to preserve list structure
    np.save(output_file, np.array(data_list, dtype=object), allow_pickle=True)
    
    print(f"✅ Saved preprocessed data to: {output_file}")
    print(f"   Load with: data = list(np.load('{output_file}', allow_pickle=True))")

# 저장 (데이터가 있는 경우)
if bsds_data is not None:
    save_preprocessed_data(bsds_data, project_root / 'data' / 'sherlock_preprocessed', roi='AG')

## 10. 요약

### 체크리스트

| Item | Status | Notes |
|------|--------|-------|
| Sherlock 데이터 다운로드 | ⬜ | Figshare에서 다운로드 필요 |
| 데이터 형태 확인 | ⬜ | (n_subjects, n_TRs, n_voxels) 예상 |
| NaN/Inf 검사 | ⬜ | 없어야 함 |
| Z-score 전처리 | ⬜ | BSDS 전처리 함수 사용 |
| BSDS 형식 변환 | ⬜ | List of (D x T) arrays |
| 피팅 테스트 | ⬜ | 소규모 테스트 후 전체 실행 |

### 서버 실행 명령

```bash
# 1. 코드 업데이트
cd /path/to/UNSEAM
git pull

# 2. BSDS 피팅 실행
python BSDS_Project/run_sherlock_bsds.py \
    --data-dir ~/data/sherlock \
    --roi AG \
    --n-states 10 \
    --preset balanced
```